# 08 - Corrective RAG (CRAG)

**Complexity:** ⭐⭐⭐⭐

**Use Cases:** High-stakes domains (legal, medical), out-of-domain queries, fact-checking

**Key Feature:** Grades document relevance, triggers web search if quality is low.

**Flow:**
```
Query → Retrieve → Grade Relevance → 
  If good: Use retrieved docs
  If poor: Web search + combine sources
```

In [17]:
import sys
sys.path.append('../..')

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from shared.config import OPENAI_VECTOR_STORE_PATH, DEFAULT_MODEL
from shared.utils import load_vector_store, print_section_header, format_docs
from shared.prompts import RELEVANCE_GRADER_PROMPT, CRAG_PROMPT
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

print_section_header("Setup: CRAG")

embeddings = OpenAIEmbeddings()
vectorstore = load_vector_store(OPENAI_VECTOR_STORE_PATH, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
llm = ChatOpenAI(model=DEFAULT_MODEL, temperature=0)

print("✅ Setup complete!")


SETUP: CRAG

✓ Loaded vector store from /Users/gianlucamazza/Workspace/notebooks/llm_rag/notebooks/advanced_architectures/../../data/vector_stores/openai_embeddings
✅ Setup complete!


## 2. Relevance Grader

In [18]:
print_section_header("Relevance Grader")

relevance_grader = RELEVANCE_GRADER_PROMPT | llm | StrOutputParser()

# Test grader
query = "What is RAG?"
relevant_doc = "RAG stands for Retrieval-Augmented Generation, a technique that..."
irrelevant_doc = "Python is a programming language used for..."

print(f"Query: '{query}'\n")
print("Testing relevance grader:")
print(f"  Relevant doc: {relevance_grader.invoke({'question': query, 'document': relevant_doc}).strip()}")
print(f"  Irrelevant doc: {relevance_grader.invoke({'question': query, 'document': irrelevant_doc}).strip()}")

print("\n✓ Relevance grader working!")


RELEVANCE GRADER

Query: 'What is RAG?'

Testing relevance grader:
  Relevant doc: yes
  Irrelevant doc: no

✓ Relevance grader working!


## 3. Web Search Tool

In [19]:
print_section_header("Web Search Tool")

try:
    from langchain_community.tools import DuckDuckGoSearchResults
    
    web_search = DuckDuckGoSearchResults(num_results=3)
    print("✓ DuckDuckGo search tool initialized")
    
    # Test
    test_results = web_search.invoke("LangChain RAG tutorial")
    print(f"\nTest search results: {test_results[:200]}...")
    
except ImportError:
    print("⚠️  DuckDuckGo search not available")
    print("   Install with: pip install duckduckgo-search")
    web_search = None


WEB SEARCH TOOL

⚠️  DuckDuckGo search not available
   Install with: pip install duckduckgo-search


## 4. CRAG Pipeline

In [20]:
print_section_header("CRAG Pipeline")

def crag_retrieve(query: str, threshold: float = 0.5):
    """CRAG retrieval with relevance grading and web fallback."""
    # Retrieve
    docs = retriever.invoke(query)
    
    # Grade relevance
    relevant_docs = []
    for doc in docs:
        grade = relevance_grader.invoke({
            "question": query,
            "document": doc.page_content[:1000]
        })
        if "yes" in grade.lower():
            relevant_docs.append(doc)
    
    relevance_ratio = len(relevant_docs) / len(docs) if docs else 0
    used_web = False
    web_results = ""
    
    # Web search if poor relevance
    if relevance_ratio < threshold and web_search:
        print(f"⚠️  Low relevance ({relevance_ratio:.0%}), using web search...")
        web_results = web_search.invoke(query)
        used_web = True
    
    context = format_docs(relevant_docs)
    if web_results:
        context += f"\n\n[WEB SEARCH]\n{web_results}"
    
    return {
        "context": context,
        "input": query,
        "used_web": used_web,
        "relevance_ratio": relevance_ratio
    }

print("✓ CRAG pipeline configured")


CRAG PIPELINE

✓ CRAG pipeline configured


## 5. CRAG Chain & Testing

In [21]:
print_section_header("CRAG Testing")

crag_chain = (
    RunnableLambda(crag_retrieve)
    | CRAG_PROMPT
    | llm
    | StrOutputParser()
)

# Test 1: In-domain (should NOT trigger web)
query1 = "What are vector stores in RAG?"
print(f"\nTest 1 (in-domain): '{query1}'")
print("=" * 80)
response1 = crag_chain.invoke(query1)
print(response1[:250])

# Test 2: Out-of-domain (should trigger web)
if web_search:
    query2 = "What is the latest Python version?"
    print(f"\n\nTest 2 (out-of-domain): '{query2}'")
    print("=" * 80)
    response2 = crag_chain.invoke(query2)
    print(response2[:250])

print("\n\n✅ CRAG adapts to document quality!")


CRAG TESTING


Test 1 (in-domain): 'What are vector stores in RAG?'
In the context of Retrieval-Augmented Generation (RAG), vector stores refer to specialized data structures or databases that store embeddings of documents or pieces of information in a high-dimensional vector space. These embeddings are typically gen


✅ CRAG adapts to document quality!


## Summary

**Advantages:**
✅ High accuracy through quality checking  
✅ Web fallback for missing info  
✅ Robust to out-of-domain queries  
✅ Transparent (shows when web used)  

**Limitations:**
- Slow (grading + potential web search)
- High cost (multiple LLM calls)
- Depends on web search quality

**When to Use:**
- High-accuracy requirements
- Out-of-domain queries expected
- Legal, medical, financial domains

**Next:** [09_self_rag.ipynb](09_self_rag.ipynb) - Self-reflective RAG